In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pickle
import time

'KOSPI200.txt' 형식으로 코스피200이 들어간 파일이 있어야함!!


In [ ]:
kp_sym_name = pd.read_csv('KOSPI200.txt'
            ,sep = "," , engine='python', encoding = 'euc-kr',
            header = None,dtype = object)
kp_sym_name.head()
kp_sym_name = kp_sym_name.rename(columns={0:"종목코드",1:"종목명"})
kp_sym_name.head()

,종목코드,종목명
0,005930,삼성전자
1,373220,LG에너지솔루션
2,000660,SK하이닉스
3,207940,삼성바이오로직스
4,035420,NAVER


In [ ]:
kp_sym = kp_sym_name["종목코드"].values.tolist()
kp_name = kp_sym_name["종목명"].values.tolist()

In [ ]:
kp_sym_name["종목코드"]==kp_sym[0]

0       True
1      False
2      False
3      False
4      False
       ...  
195    False
196    False
197    False
198    False
199    False
Name: 종목코드, Length: 200, dtype: bool

코스피50 판매량 추출

In [ ]:
from tqdm import tqdm

for i in range(50): 
  kospi_box = []
  for n in tqdm(range(1,19)): # 약 1년치 판매량 
    url = requests.get("https://finance.naver.com/item/frgn.naver?code={}&page={}".format(kp_sym[i],n),
                        headers = {"User-Agent":"Mozilla/5.0"})
    html = BeautifulSoup(url.text)
    table = html.find_all("table", class_ = "type2")[1]
    table = pd.read_html(str(table))[0]
    table.columns = table.columns.droplevel(0)
    table = table[['날짜','거래량']].dropna()
    
    kospi_box.append(table)
    time.sleep(1)

  kospi_box = pd.concat(kospi_box,ignore_index = True)
  kospi_box.rename(columns = {'날짜':'날짜','거래량':kp_name[i]},inplace = True)

  if i == 0:
      kospi_total = kospi_box
      
  if i != 0:
      kospi_total = kospi_total.merge(kospi_box, on = '날짜')
      
kospi_total 

 83%|████████▎ | 15/18 [00:31<00:06,  2.17s/it]

LG에너지 상장 날 수로인한 데이터 중복.  
이를 제거해야함

In [ ]:
kospi_total.set_index('날짜',inplace = True)
kospi_total = kospi_total.loc[~kospi_total.index.duplicated(keep='first')] #LG에너지는 22.01.27부터 상장해서 duplicate됨.이를 삭제 필요
kospi_total.sort_index(ascending=True,inplace = True)
kospi_total

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:6402: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  key,


,삼성전자,LG에너지솔루션
날짜,,
2022.01.27,22274777.0,15946992.0
2022.01.28,21367447.0,4559773.0
2022.02.03,17744721.0,2918435.0
2022.02.04,12730034.0,2088996.0
2022.02.07,14240838.0,1911176.0
...,...,...
2022.07.22,10261310.0,300672.0
2022.07.25,9193681.0,653865.0
2022.07.26,6597211.0,691524.0


In [ ]:
import pandas as pd

In [ ]:
# 판매량 하나의 파일로 합치기 
file_total = pd.read_csv("kosoi_box_{}.csv".format(kp_name[0]))
file_total.rename(columns = {'날짜':'날짜','거래량':kp_name[0]},inplace = True)

for i in range(1,50):
  temp = pd.read_csv("kosoi_box_{}.csv".format(kp_name[i]))
  temp.rename(columns = {'날짜':'날짜','거래량':kp_name[i]},inplace = True)
  file_total = file_total.merge(temp, on ='날짜')
file_total.set_index('날짜',inplace = True)
file_total = file_total.loc[~file_total.index.duplicated(keep='first')] #LG에너지는 22.01.27부터 상장해서 duplicate됨.이를 삭제 필요
file_total.sort_index(ascending=True,inplace = True)